In [3]:
pip install spacy

     --------------------------------------- 12.2/12.2 MB 31.2 MB/s eta 0:00:00
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
     ---------------------------------------- 482.8/482.8 kB ? eta 0:00:00
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached pathy-0.10.1-py3-none-any.whl (48 kB)
     ---------------------------------------- 1.5/1.5 MB 47.3 MB/s eta 0:00:00
  Using cached typer-0.7.0-py3-none-any.whl (38 kB)
  Using cached spacy_loggers-1.0.4-py3-none-any.whl (11 kB)
  Using cached wasabi-1.1.1-py3-none-any.whl (27 kB)
     ---------------------------------------- 2.2/2.2 MB 35.0 MB/s eta 0:00:00
     ---------------------------------------- 96.8/96.8 kB 5.4 MB/s eta 0:00:00
  Using cached catalogue-2.0.8-py3-none-any.whl (17 kB)
  Using cached confection-0.0.4-py3-none-any.whl (32 kB)
     ---------------------------------------- 7.0/7.0 MB 40.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

                                              0.0/11.2 MB ? eta -:--:--
                                              0.0/11.2 MB ? eta -:--:--
                                             0.0/11.2 MB 325.1 kB/s eta 0:00:35
                                             0.0/11.2 MB 325.1 kB/s eta 0:00:35
                                             0.1/11.2 MB 363.1 kB/s eta 0:00:31
                                             0.1/11.2 MB 554.9 kB/s eta 0:00:20
                                             0.2/11.2 MB 655.4 kB/s eta 0:00:17
                                             0.3/11.2 MB 896.4 kB/s eta 0:00:13
     -                                        0.4/11.2 MB 1.0 MB/s eta 0:00:11
     -                                        0.5/11.2 MB 1.3 MB/s eta 0:00:09
     --                                       0.6/11.2 MB 1.4 MB/s eta 0:00:08
     --                                       0.7/11.2 MB 1.5 MB/s eta 0:00:07
     --                                       0.8/11.2 MB 1.

In [4]:
pip install spacytextblob


  Using cached spacytextblob-4.0.0-py3-none-any.whl (4.5 kB)
  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)


In [5]:
!python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 17.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [6]:
!pip install bs4
!pip install gensim

  Using cached bs4-0.0.1.tar.gz (1.1 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=e76573f216eecc7567049b45381fbceed723a8595c853f002e864ef3549c3dd4
  Stored in directory: c:\users\biehl\appdata\local\pip\cache\wheels\8b\ac\c5\cb646ab01df6f353a9994b1c97dffb94f9b12013d64c162c58
Successfully built bs4


In [1]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
# Loading our two pipelines
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

config = {"punct_chars": [".", "?", "!", "。"]}
nlp.add_pipe('sentencizer', config=config)

In [3]:
# Using panda to load in our .csv file
df = pd.read_csv("urls.csv") # File we are checking
urls = df["Address"].tolist() # Column name we are checking
url_sent_score = []
url_sent_label = []
url_subj_score = []
url_subj_label = []
total_pos = []
total_neg = []

In [4]:
def get_named_entities(doc):
    ent_list = []
    for ent in doc.ents:
        ent_list.append((ent.text,ent.label_))
    return ent_list

def get_sentences(doc):
    return doc.sents

def get_sentence_list(doc):
    return [sent for sent in doc.sents]

def sentence_sentiment_from_doc(doc):
    sentences = get_sentences(doc)
    tuple_list = []
    for sentence in sentences:
        sent_doc = nlp(sentence.text)
        tuple_list.append((sentence,sent_doc._.blob.polarity))
    return tuple_list

#def sentence_sentiment_on_named_entities(doc):
    

In [5]:
#https://pypi.org/project/spacytextblob/
for count, x in enumerate(urls):
    try:
        url = x

        headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
        res = requests.get(url,headers=headers)
        html_page = res.text

        soup = BeautifulSoup(html_page, 'html.parser')
        for script in soup(["script", "style","meta","label","header","footer"]):
            script.decompose()
        page_text = (soup.get_text()).lower()
        page_text = page_text.strip().replace("  ","")
        page_text = "".join([s for s in page_text.splitlines(True) if s.strip("\r\n")])

        # Start the sentiment analysis now
        doc = nlp(page_text)
        sentiment = doc._.blob.polarity
        sentiment = round(sentiment,2)
        subjectivity = doc._.blob.subjectivity
        subjectivity = round(subjectivity,2)
        
        #exploration of single topic modeling and sentiment
        #print(doc._.blob.sentiment_assessments)
        #print([(w.text, w.pos_) for w in doc])
        
        if count / 1 == 1:
            print("enter test spot\n")
            ent_list = get_named_entities(doc)
            #print(ent_list)
            sentence_sentiment_list = sentence_sentiment_from_doc(doc)
            print(sentence_sentiment_list)
            break
            
        #print("\n")

        # Gives positive or negative label
        if sentiment >= -0.033 and sentiment <= 0.043:
            sent_label = "Neutral"
        elif sentiment > 0.043 and sentiment < 0.143:
            sent_label = "Neutral Positive"
        elif sentiment > 0.143:
            sent_label = "Positive"
        elif sentiment < -0.033 and sentiment > -0.062:
            sent_label = "Neutral Negative"
        elif sentiment < -0.062:
            sent_label = "Negative"

        url_sent_label.append(sent_label)
        url_sent_score.append(sentiment)
        url_subj_score.append(subjectivity)

        positive_words = []
        negative_words = []

        for x in doc._.blob.sentiment_assessments.assessments:
          if x[1] > 0:
            positive_words.append(x[0][0])
          elif x[1] < 0:
            negative_words.append(x[0][0])
          else:
            pass

        total_pos.append(', '.join(set(positive_words)))
        total_neg.append(', '.join(set(negative_words)))
    except:
        url_sent_label.append("Error")
        url_sent_score.append(0.0)
        url_subj_score.append(0.0)

        positive_words = []
        negative_words = []

        total_pos.append(', '.join(set(positive_words)))
        total_neg.append(', '.join(set(negative_words)))

# print("The average sentiment score was: " + str(sum(url_sent_score) / len(url_sent_score)))
# print("The average subjectivity score was: " + str(sum(url_subj_score) / len(url_subj_score)))

# df["Sentiment Score"] = url_sent_score
# df["Sentiment Label"] = url_sent_label
# df["Subjectivity Score"] = url_subj_score
# df["Positive Words"] = total_pos
# df["Negative Words"] = total_neg

# df

enter test spot

[(11 minutes of aerobics daily lowers disease risk, study says | cnn
 
 
11 minutes of daily exercise could have a positive impact on your health, large study shows
			by kristen rogers, cnn
		
updated
6:45 am est, wed march 1, 2023
link copied!
 
, 0.12378246753246752), (this may look silly, but a new study reveals its benefits to your daily routine
02:34
 - source:
cnn
your health
16 videos
this may look silly, but a new study reveals its benefits to your daily routine
02:34
now playing
 - source:
, -0.12121212121212122), (cnn
if you have allergies, you may want to avoid this type of medication
02:18
now playing
 - source:
cnn
doctor explains new over-the-counter medicine that's a 'complete antidote to opioids'
01:59
now playing
 - source:
cnn
maternity wards across the us are closing because of this issue
02:31
now playing
 - source:
cnn
nearly 40% of dementia cases can be prevented with one small health change
02:13
now playing
 - source:
cnn
this type of exercise 

In [6]:
x = get_sentence_list(doc)



In [98]:
import gensim
from gensim.models import FastText
from gensim import corpora, models
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import numpy as np
import os
from random import shuffle
import re
import numpy as np
import fasttext
import fasttext.util


In [99]:
input_text = []
input_text = [str(sentence) for sentence in x]


In [28]:
fasttext.util.download_model('en', if_exists='ignore')  # English

'cc.en.300.bin'

In [39]:
ft_model = fasttext.load_model('cc.en.300.bin')

In [105]:
# store as list of lists of words
sentences_ted = []
for sent_str in input_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

stoplist = []
with open('stopwords') as openfileobject:
    for line in openfileobject:
        stoplist.append(line[:-1])
stoplist = set(stoplist)

for i in range(len(sentences_ted)):
    for word in reversed(sentences_ted[i]):
        if word in stoplist:
            sentences_ted[i].remove(word)

In [107]:
sentences_ted

[['11',
  'minutes',
  'aerobics',
  'daily',
  'lowers',
  'disease',
  'risk',
  'study',
  'says',
  'cnn',
  '11',
  'minutes',
  'daily',
  'exercise',
  'could',
  'positive',
  'impact',
  'health',
  'large',
  'study',
  'shows',
  'kristen',
  'rogers',
  'cnn',
  'updated',
  '6',
  '45',
  'est',
  'wed',
  'march',
  '1',
  '2023',
  'link',
  'copied'],
 ['may',
  'look',
  'silly',
  'new',
  'study',
  'reveals',
  'benefits',
  'daily',
  'routine',
  '02',
  '34',
  'source',
  'cnn',
  'health',
  '16',
  'videos',
  'may',
  'look',
  'silly',
  'new',
  'study',
  'reveals',
  'benefits',
  'daily',
  'routine',
  '02',
  '34',
  'playing',
  'source'],
 ['cnn',
  'allergies',
  'may',
  'want',
  'avoid',
  'type',
  'medication',
  '02',
  '18',
  'playing',
  'source',
  'cnn',
  'doctor',
  'explains',
  'new',
  'counter',
  'medicine',
  'complete',
  'antidote',
  'opioids',
  '01',
  '59',
  'playing',
  'source',
  'cnn',
  'maternity',
  'wards',
  'acros

In [108]:
#model = FastText(vector_size=500, window=2, min_count=1)  # instantiate
#model.build_vocab(corpus_iterable=sentences_ted)
#model.train(corpus_iterable=sentences_ted, total_examples=len(sentences_ted), epochs=50)  # train
num_topics = 10
bow_corpus = [dictionary.doc2bow(text) for text in sentences_ted]
dictionary = Dictionary(sentences_ted)
dictionary.filter_extremes(no_below=0.1, no_above=0.9)
lda_model = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=10)



IndexError: index 474 is out of bounds for axis 1 with size 474

In [104]:
file = open("out.txt", "w")
for topic_id in range(num_topics):
    file.write(f"Topic {topic_id + 1}:\n")
    for term, weight in lda_model.show_topic(topic_id):
        file.write(f"\t{term}: {weight}\n")

In [50]:
model.wv.most_similar("students")

NameError: name 'model' is not defined

In [ ]:
model.wv.similarity('system', 'time')

In [14]:
topics = model.show_topics(num_topics=-1, formatted=False)

AttributeError: 'FastText' object has no attribute 'show_topics'